El objetivo es trucar la red neuronal **Inception V3** con un ataque adversario.

In [2]:
import tensorflow as tf # plataforma para el aprendizaje automatico 
import keras # deep learning framework/api

import matplotlib.pyplot as plt
import numpy as np

In [3]:
from keras.applications.inception_v3 import InceptionV3, decode_predictions # importamos la red neuronal ya entrenada
from keras import backend as k

In [6]:
iv3 = InceptionV3() # Cargamos la red neuronal, la primera vez se descarga de internet

In [26]:
# print(iv3.summary()) # Ver resumen de la red neuronal

from keras.preprocessing import image

# Para introducir la imágen a InceptionV3, la imagen requiere de unos requisitos
# Cargamos la imagen con dimension 299x299 (para iv3) y la convertimos a una matriz de datos
coche_2 = image.img_to_array(image.load_img("./img/coche_2.jpg", target_size=(299,299)))

# Cambiamos el rango o base, de [0-255] a [-1-1], por que
# así espera iv3 que pases los datos de la imágen
coche_2 /= 255
coche_2 -= 0.5
coche_2 *= 2

# La red neuronal espera que como entrada le pasemos un tensor,
# este tiene una dimensión más que la imágen que le pasamos,
# la primera dimensión hace referencia al tamaño del lote o "badge size?"
# esto es útil si queremos meter varias imágenes

coche_2 = coche_2.reshape([1, coche_2.shape[0], coche_2.shape[1], coche_2.shape[2]])

# LISTO, la variable ya esta lista para pasarsela al modelo
resultado = iv3.predict(coche_2)
print(decode_predictions(resultado))

[[('n04285008', 'sports_car', 0.8140701), ('n03459775', 'grille', 0.089450285), ('n02974003', 'car_wheel', 0.042520903), ('n03100240', 'convertible', 0.016252259), ('n04037443', 'racer', 0.0022947474)]]
